In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import torchvision
from torch.autograd import Variable
from tqdm import tqdm_notebook

In [ ]:
class MnistModel(nn.Module):
    def __init__(self):
        super(MnistModel, self).__init__()
        # input is 28x28
        # padding=2 for same padding
        self.conv1 = nn.Conv2d(3, 32, 5, stride=1, padding=2)
        # feature map size is 14*14 by pooling
        # padding=2 for same padding
        self.conv2 = nn.Conv2d(32, 64, 5, stride=1, padding=2)
        # feature map size is 7*7 by pooling
        self.conv3 = nn.Conv2d(64, 64, 5, stride=1, padding=2)
        self.fc1 = nn.Linear(64*28*50, 512)
        self.fc2 = nn.Linear(512, 10)
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = F.max_pool2d(F.relu(self.conv3(x)), 2)
        x = x.view(-1, 64*28*50)   # reshape Variable
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)
    
model = MnistModel()
model

In [ ]:
batch_size=2
n_threads=3

transform = transforms.Compose([
    #transforms.CenterCrop(10),
    transforms.ToTensor(),
])

imagenet_data = torchvision.datasets.ImageFolder('./data/trainImg',transform=transform)
train_loader = torch.utils.data.DataLoader(imagenet_data,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          num_workers=n_threads)

In [ ]:
#test_loader = torch.utils.data.DataLoader(
#    datasets.MNIST('data', train=False, transform=transforms.ToTensor()),
#    batch_size=1000)

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
model.train()
train_loss = []
train_accu = []
i = 0
model.cuda()
for epoch in range(15):
    for data, target in tqdm_notebook(train_loader):
        data, target = Variable(data), Variable(target)
        data = data.cuda()
        target = target.cuda()
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()    # calc gradients
        train_loss.append(loss.data[0])
        optimizer.step()   # update gradients
        prediction = output.data.max(1)[1]   # first column has actual prob.
        accuracy = prediction.eq(target.data).sum()/batch_size*100
        train_accu.append(accuracy)
        if i % 100 == 0:
            print('Train Step: {}\tLoss: {:.3f}\tAccuracy: {:.3f}'.format(i, loss.data[0], accuracy))
        i += 1